# Creating a Police Database

##### Using the Police UK API (Home | data.police.uk), extract the data related to street-level crime and outcome data as well as the nearest police stations. Create a single database that will contain all the street level crime and outcome data with the nearest police station that have investigated the crimes and remove all the data that have a missing outcome while highlighting which authorities do not provide an outcome.

## Loading the Data

##### Here I will need to use the .get() function to retrieve the data from the API. Reading through the documentation I have found that the data is kept in a JSON format, so I expect to use the requests module to send HTTP requests and the json library to work with the data. 

In [1]:
import requests 
import json

crimes_API = requests.get('https://data.police.uk/api/crimes-street/all-crime')
test_API = requests.get('https://data.police.uk/api/crimes-street/all-crime?lat=52.629729&lng=-1.131592&date=2017-01')
print(crimes_API.status_code)
print(test_API.status_code)

400
404


#### The error code 400 either means that there was a bad request, or according to the documentation that the request length is over 4094 characters (which it is not). So I have tried using the example request also, and this has thrown a 404 not found error, the same result given when I tried the link in-browser. I need to try some testing to see if other methods or endpoints to see if I can retrieve what I need another way.

In [2]:
import pandas as pd 

forces = pd.read_json('https://data.police.uk/api/forces')
crime_cat = pd.read_json('https://data.police.uk/api/crime-categories')

cat_exts = crime_cat['url']

# crime_data = pd.read_json('https://data.police.uk/api/crimes-street/burglary') this did not work, implying I cannot retrieve any street-crime data

print(forces.head(), '\n\n') # print the first 5 forces and a couple of empty lines
print(crime_cat.head()) # print the first 5 crime categories


                  id                            name
0  avon-and-somerset  Avon and Somerset Constabulary
1       bedfordshire             Bedfordshire Police
2     cambridgeshire     Cambridgeshire Constabulary
3           cheshire           Cheshire Constabulary
4     city-of-london           City of London Police 


                     url                       name
0              all-crime                  All crime
1  anti-social-behaviour      Anti-social behaviour
2          bicycle-theft              Bicycle theft
3               burglary                   Burglary
4  criminal-damage-arson  Criminal damage and arson


#### Some endpoints still work, but not the main one I need. Other options are finding a different API to use, this would be the best option but I have been told specifically to use this API, or download the data and use it locally. To overcome this roadblock I will download the required data locally and continue.

#### After downloading the criminal data and investigating, I will begin combining the data into one large database.

In [3]:
import os
rootdir = 'Crime Data'

outcome_paths = [] # I will keep all of the file paths for the outcome data here
crime_paths = [] # I will keep all the paths for the main crime data here

# Here I need to loop through all the date folder and within them, the location files for both
# outcome and crime data. 
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('outcomes.csv'): # identify outcome files
            outcome_paths.append('Crime Data' + '/' + file[:7] + '/' + file)
        elif file.endswith('street.csv'): # identify main files
            crime_paths.append('Crime Data' + '/' + file[:7] + '/' + file)
        else:
            continue

# Now to use the lists with all the paths in them to create two data frames.
outcome_df = pd.concat(map(pd.read_csv, outcome_paths), ignore_index=True) # map the read_csv fnc to paths
crime_df = pd.concat(map(pd.read_csv, crime_paths), ignore_index=True)
            

#### I have loaded the data and separated it into the outcome and main crime data. Now to do some initial analysis and decide what I need in my final database to avoid repeated information, duplicates and irrelevant data to the task.

In [4]:
print('Outcome Information', '\n')
print(outcome_df.info())

Outcome Information 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13177233 entries, 0 to 13177232
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Crime ID      object 
 1   Month         object 
 2   Reported by   object 
 3   Falls within  object 
 4   Longitude     float64
 5   Latitude      float64
 6   Location      object 
 7   LSOA code     object 
 8   LSOA name     object 
 9   Outcome type  object 
dtypes: float64(2), object(8)
memory usage: 1005.3+ MB
None


In [5]:
print('Crime Information', '\n')
print(crime_df.info())


Crime Information 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18899067 entries, 0 to 18899066
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Crime ID               object 
 1   Month                  object 
 2   Reported by            object 
 3   Falls within           object 
 4   Longitude              float64
 5   Latitude               float64
 6   Location               object 
 7   LSOA code              object 
 8   LSOA name              object 
 9   Crime type             object 
 10  Last outcome category  object 
 11  Context                float64
dtypes: float64(3), object(9)
memory usage: 1.7+ GB
None


In [6]:
outcome_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Outcome type
0,aa36cff6a311dc3781080161311195c51a0e21d0ab8394...,2020-01,Suffolk Constabulary,Suffolk Constabulary,NaN,NaN,No location,NaN,NaN,Investigation complete; no suspect identified
1,63630c5dcef2a5cd6cbe2d595d56744cb5148d8b34d392...,2020-01,Suffolk Constabulary,Suffolk Constabulary,NaN,NaN,No location,NaN,NaN,Investigation complete; no suspect identified
2,3e3982fe4d6d4d51f33de2ace9e30046134a39a3319fee...,2020-01,Suffolk Constabulary,Suffolk Constabulary,NaN,NaN,No location,NaN,NaN,Investigation complete; no suspect identified
3,fb4768c340f7c75a203ce04a128bf4d8f9c658825f6568...,2020-01,Suffolk Constabulary,Suffolk Constabulary,NaN,NaN,No location,NaN,NaN,Suspect charged
4,4d74df4720b0039a01beb98822c2dbdf1be0208dc8170d...,2020-01,Suffolk Constabulary,Suffolk Constabulary,NaN,NaN,No location,NaN,NaN,Offender given a caution


In [7]:
crime_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,c7000c41002f19263d4adec66b911f1c3f5e7eeb1302a3...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.570572,53.607792,On or near Park/Open Space,E01007418,Barnsley 016A,Other theft,Status update unavailable,NaN
1,b8bc1b6cf423a9431734982fffb11f803cf82140702cc7...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.670108,53.553629,On or near Huddersfield Road,E01007426,Barnsley 027D,Robbery,Investigation complete; no suspect identified,NaN
2,8c69cefea36edafe5fa3f992ccc31d3cfd0c9af9a81429...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.879031,53.943807,On or near Cross End Fold,E01010646,Bradford 001A,Bicycle theft,Investigation complete; no suspect identified,NaN
3,7236a8fb307214df61ca7a9b93bd309c0524f9bee58d89...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.882746,53.933807,On or near Cocking Lane,E01010646,Bradford 001A,Burglary,Investigation complete; no suspect identified,NaN
4,92803cebdfde14ad870899c45c6bee398331f33a053bf1...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.881467,53.944510,On or near Kilners Croft,E01010646,Bradford 001A,Criminal damage and arson,Investigation complete; no suspect identified,NaN


#### The task requires me to gather the information from all street level crime and outcome data, along with the nearest police stations. This means I will need the Crime ID, Month, Reported By, Falls within, Location, Crime Type, Last outcome category from the crime_df, then from the outcome_df I will need the Outcome type. I'll use the primary key of the Crime ID to match the correct outcome types to the appropriate crime in the new database. I can create my table in SQL as below, or I can do it with the pandas library.

In [8]:
import sqlite3

conn = sqlite3.connect('example_sql_db') 
c = conn.cursor()

# Now to create the tables in the database
c.execute('''
          CREATE TABLE IF NOT EXISTS station_outcomes
          (
          [station] TEXT PRIMARY KEY, 
          [no_outcomes] INTEGER
          )
          ''')
          
c.execute('''
          CREATE TABLE IF NOT EXISTS crime_data
          (
          [crime_id] TEXT PRIMARY KEY, 
          [month] TEXT,
          [reported_by] TEXT,
          [falls_within] TEXT,
          [location] TEXT,
          [crime_type] TEXT,
          [last_outcome_category] TEXT,
          [outcome_type] TEXT
          )
          ''')

conn.commit()

#### Now to start organizing the data to into dataframes that I can then load into the tables I have created.

In [9]:
# Filter the crimes and outcome dataframes to only include the columns I need
final_df = crime_df.filter(['Crime ID', 'Month', 'Reported by', 'Falls within', 'Location', 'Crime type', 'Last outcome category'], axis=1)
outcome_df = outcome_df.filter(['Crime ID', 'Outcome type'])

# Left merge the outcome_df and the final_df, then I can filter to just include the columns I need.
final_df = final_df.merge(outcome_df, on=['Crime ID'], how='left') 

#### Now to fill the station outcomes table. For this I will count the amount of null outcomes for each station, then store them against that station in a table. To identify true missing outcomes I would like to count the number of nulls in the outcome columns of my final dataframe.

In [10]:
empty_df = final_df[final_df['Outcome type'].isna()]

empty_df[empty_df['Last outcome category'].notnull()].head()

,Crime ID,Month,Reported by,Falls within,Location,Crime type,Last outcome category,Outcome type
0,c7000c41002f19263d4adec66b911f1c3f5e7eeb1302a3...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Park/Open Space,Other theft,Status update unavailable,NaN
5,930c29edbf799c97c2272515cefebc7226e408edbc3dec...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Parking Area,Criminal damage and arson,Status update unavailable,NaN
8,b654782b72ef41536a80f76b30e09b6ad27bd02e4da464...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Moorside Lane,Other theft,Status update unavailable,NaN
17,7b4fb7cc3bfb2c88e1689077c560577a7405736ffe2e1b...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Sefton Drive,Violence and sexual offences,Status update unavailable,NaN
23,c46962d275f9db2f94d9b555144a66898319b42d79d256...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Wheatley Road,Burglary,Status update unavailable,NaN


#### The majority of the differences appear to be because the Last Outcome Category is 'Status update unavailable'. So in i will make a new column to make sure any crimes without outcomes are type null instead of Status update unavailable.

In [11]:
import numpy as np

final_df.loc[final_df['Last outcome category'] == 'Status update unavailable', 'Last outcome category'] = np.nan

In [12]:
final_df['Outcome'] = final_df['Last outcome category'] if not np.nan else final_df['Outcome type']
final_df.head()

,Crime ID,Month,Reported by,Falls within,Location,Crime type,Last outcome category,Outcome type,Outcome
0,c7000c41002f19263d4adec66b911f1c3f5e7eeb1302a3...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Park/Open Space,Other theft,NaN,NaN,NaN
1,b8bc1b6cf423a9431734982fffb11f803cf82140702cc7...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Huddersfield Road,Robbery,Investigation complete; no suspect identified,Investigation complete; no suspect identified,Investigation complete; no suspect identified
2,8c69cefea36edafe5fa3f992ccc31d3cfd0c9af9a81429...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Cross End Fold,Bicycle theft,Investigation complete; no suspect identified,Investigation complete; no suspect identified,Investigation complete; no suspect identified
3,7236a8fb307214df61ca7a9b93bd309c0524f9bee58d89...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Cocking Lane,Burglary,Investigation complete; no suspect identified,Investigation complete; no suspect identified,Investigation complete; no suspect identified
4,92803cebdfde14ad870899c45c6bee398331f33a053bf1...,2020-01,West Yorkshire Police,West Yorkshire Police,On or near Kilners Croft,Criminal damage and arson,Investigation complete; no suspect identified,Investigation complete; no suspect identified,Investigation complete; no suspect identified


#### Now I have collated all the data into one dataframe which I can use to count the number of null outcomes for each station location. So I can now delete the last outcome category and outcome type columns.

In [13]:
del final_df['Last outcome category']
del final_df['Outcome type']

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19872722 entries, 0 to 19872721
Data columns (total 7 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Crime ID      object
 1   Month         object
 2   Reported by   object
 3   Falls within  object
 4   Location      object
 5   Crime type    object
 6   Outcome       object
dtypes: object(7)
memory usage: 1.2+ GB


In [14]:
outcome_counts_df = final_df.filter(['Falls within', 'Outcome'])

stations = outcome_counts_df['Falls within'].unique()
null_count = []

for station in stations:
    series = outcome_counts_df.loc[outcome_counts_df['Falls within'] == station]
    null_count.append(series['Outcome'].isna().sum())

counts_df = pd.DataFrame({'Stations': stations, 'No outcome counts': null_count})


In [15]:
counts_df.head()

,Stations,No outcome counts
0,West Yorkshire Police,181399
1,Humberside Police,62320
2,Hertfordshire Constabulary,101168
3,Gloucestershire Constabulary,78871
4,Cleveland Police,80736


#### Now we have a dataframe of all the stations and their crimes with no outcomes. All that is left to do is to remove all null values and load them into sqlite databases.

In [16]:
# Remove the crime df rows where the outcome columns is NaN
crime_df = final_df.dropna(subset=['Outcome'])

con = sqlite3.connect('final_crime_db')

# Write the new DataFrame to a new SQLite table
crime_df.to_sql("crime_db", con, if_exists="replace")

con.close()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [17]:
con = sqlite3.connect('final_no_outcome_db')

# Write the new DataFrame to a new SQLite table
counts_df.to_sql("counts_db", con, if_exists="replace")

con.close()

In [23]:
counts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Stations           45 non-null     object
 1   No outcome counts  45 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 848.0+ bytes


#### I'll also save them to csv files to be sure that my result can be read easily.

In [29]:
condensed = crime_df.sample(n = 100)
condensed.to_csv('crime.csv', index=False)
counts_df = counts_df.sort_values(by = 'No outcome counts')
counts_df.to_csv('station_no-outcome_count.csv', index=False)

#### With more time I would like to compare and reconsile the rest of the data, while also adding another column that classifies the counts indicating whether they are good or bad in comparison to the average.